# Handling imbalanced data sheet with Machine Learning
  


In [51]:
import pandas as pd
import numpy as np
#https://www.kaggle.com/mlg-ulb/creditcardfraud/activity

In [52]:
pip install imbalanced-learn 

In [53]:
df=pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [54]:
df.shape

(29799, 31)

In [57]:
#df.isnull().sum()
## dropping the null values as from the above command we found that some of the values are missing
df=df.dropna()

In [60]:
#df.isnull().sum()
df.shape

(29798, 31)

In [61]:
df['Class'].value_counts()
# to detect if it is a fraud or not
## veryvery huge difference between frauds (1) and not frauds(0)

0.0    29704
1.0       94
Name: Class, dtype: int64

In [62]:

## independenet and dependent values
X=df.drop("Class",axis=1)
y= df.Class

## Cross-validation like KFold and Hyperparameter tuning 

In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import GridSearchCV


In [64]:
log_class = LogisticRegression()
## classifier (above)
#hyper parameter tuning - ML playlist
grid = {'C': 10.0**np.arange(-2,3),'penalty':['l1','l2']}
cv=KFold(n_splits=5, random_state=None, shuffle=False)

In [65]:
## Train - test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.7)

In [66]:
clf=GridSearchCV(log_class,grid,cv=cv, n_jobs=-1,scoring='f1_macro')

In [67]:
clf.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_macro', verbose=0)

In [68]:
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

##for imbalance data set if you are getting a better accuracy, there is something wrong.
## accuracy score high, doesnt mean model is good. Look into other features: precision, recall, other performance matrices

[[8901   15]
 [   5   19]]
0.9977628635346756
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      8916
         1.0       0.56      0.79      0.66        24

    accuracy                           1.00      8940
   macro avg       0.78      0.89      0.83      8940
weighted avg       1.00      1.00      1.00      8940



In [72]:
class_weight=dict({0:1,1:100})
##for 0 the weight is 1  i.e. value doesnt change
## for 1, the weight is 100, so new value = old *100

In [73]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(class_weight=class_weight)
classifier.fit(X_train,y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 1, 1: 100}, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [74]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

##decision trees will not get much impact even if data is imbalanced set
## if we apply Cross validation and hyper-tuning , results will improve
## now we apply a hyper tuning, class_weight

## hyperparameter tuning: "live project playlists"

[[8913    3]
 [   2   22]]
0.9994407158836689
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      8916
         1.0       0.88      0.92      0.90        24

    accuracy                           1.00      8940
   macro avg       0.94      0.96      0.95      8940
weighted avg       1.00      1.00      1.00      8940



#### Under sampling
By using undersampling, we reduce the points of the maximum labels
- Disadv: Huge loss of data
- to be used in scenarios where we have very less data set


In [76]:
from collections import Counter
from imblearn.under_sampling import NearMiss
ns= NearMiss(0.80)
X_train_ns, y_train_ns=ns.fit_sample(X_train,y_train)
print("The number of class before the fit {}".format(Counter(y_train)))
print("The number of class after the fit {}".format(Counter(y_train_ns)))

## in near miss, we decarease the value of the feature having more elements in such a way that
## 0.8 * the total (new number of features)  = old number with less number of feature

The number of class before the fit Counter({0.0: 20788, 1.0: 70})
The number of class after the fit Counter({0.0: 87, 1.0: 70})


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

In [77]:
0.8*87

69.60000000000001

In [78]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [79]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
## precision is GONE
## never do UNDERSAMPLING unless your data is very very less

[[8288  628]
 [   1   23]]
0.9296420581655481
              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96      8916
         1.0       0.04      0.96      0.07        24

    accuracy                           0.93      8940
   macro avg       0.52      0.94      0.52      8940
weighted avg       1.00      0.93      0.96      8940



## Over Sampling

In [80]:
from imblearn.over_sampling import RandomOverSampler

In [85]:
#os= RandomOverSampler(0.50)
os= RandomOverSampler(0.750)
X_train_os, y_train_os=os.fit_sample(X_train,y_train)
print("The number of class before the fit {}".format(Counter(y_train)))
print("The number of class after the fit {}".format(Counter(y_train_os)))


## in oversampleing, we increase the value of the feature having mless elements in such a way that
## 0.5 * the old feature with more number  = new increased number with less number of feature

The number of class before the fit Counter({0.0: 20788, 1.0: 70})
The number of class after the fit Counter({0.0: 20788, 1.0: 15591})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [86]:
20788*0.5

10394.0

In [87]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train_os,y_train_os)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [88]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[8913    3]
 [   1   23]]
0.9995525727069351
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      8916
         1.0       0.88      0.96      0.92        24

    accuracy                           1.00      8940
   macro avg       0.94      0.98      0.96      8940
weighted avg       1.00      1.00      1.00      8940



###SMOTETomek


In [89]:
from imblearn.combine import SMOTETomek  

In [91]:
os= SMOTETomek(0.750)
X_train_os, y_train_os=os.fit_sample(X_train,y_train)
print("The number of class before the fit {}".format(Counter(y_train)))
print("The number of class after the fit {}".format(Counter(y_train_os)))
## SMOTETomek creates new points for the lowest number  (based on the nearest points, more points are created)
## Takes more time than oversampling


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


The number of class before the fit Counter({0.0: 20788, 1.0: 70})
The number of class after the fit Counter({0.0: 20696, 1.0: 15499})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [92]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train_os,y_train_os) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [93]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[8910    6]
 [   1   23]]
0.9992170022371365
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      8916
         1.0       0.79      0.96      0.87        24

    accuracy                           1.00      8940
   macro avg       0.90      0.98      0.93      8940
weighted avg       1.00      1.00      1.00      8940



## Ensemble techniques


In [94]:
from imblearn.ensemble import EasyEnsembleClassifier  


In [96]:
easy=EasyEnsembleClassifier()
easy.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

EasyEnsembleClassifier(base_estimator=None, n_estimators=10, n_jobs=1,
                       random_state=None, replacement=False,
                       sampling_strategy='auto', verbose=0, warm_start=False)

In [97]:
y_pred = easy.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[8710  206]
 [   1   23]]
0.9768456375838926
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99      8916
         1.0       0.10      0.96      0.18        24

    accuracy                           0.98      8940
   macro avg       0.55      0.97      0.59      8940
weighted avg       1.00      0.98      0.99      8940



To be continued in next lecture....